In [232]:
import time

In [233]:
class TreeNode(object):
    def __init__(self, left=None, right=None, value=None):
        self.left = left
        self.right = right
        self.value = value

In [234]:
class Tree(object):
    def __init__(self, root=None):
        self.root = root

    def inorder(self):
        def traverse(node):
            travel = []
            if node.left != None:
                travel = traverse(node.left)
            travel.append(node.value)
            if node.right != None:
                travel += traverse(node.right)
            return travel

        return traverse(self.root)

    def preorder(self):
        def traverse(node):
            travel = []
            travel.append(node.value)
            if node.left != None:
                travel += traverse(node.left)
            if node.right != None:
                travel += traverse(node.right)
            return travel

        return traverse(self.root)

    def postorder(self):
        def traverse(node):
            travel = []
            travel.append(node.value)
            if node.left != None:
                travel += traverse(node.left)
            if node.right != None:
                travel += traverse(node.right)
            return travel

        return traverse(self.root)

In [235]:
class TreeGenerator(Tree):
    def __init__ (self, list, type="btree"):
        self.list = list
        
        if type == "btree":
            t1 = time.time()
            self.build_btree_from_list(list)
            print(time.time()-t1)
        
    def build_btree_from_list(self, arr):
        # def insert(node, arr, processed, start, end):
        #     mid = (end - start)//2 + start
            
        #     if mid in processed:
        #         return
        #     if node.value != None:
        #         if arr[mid] <= node.value:
        #             node.left = TreeNode(value=arr[mid])
        #             node = node.left
        #         else:
        #             node.right = TreeNode(value=arr[mid])
        #             node = node.right
        #     else:
        #         node.value = arr[mid]
                
        #     print(arr[mid], start, mid, end)
            
        #     processed.append(mid)
        #     insert(node, arr, processed, start, mid)
        #     insert(node, arr, processed, mid, end)
        def insert(node, arr):
            if len(arr) == 0: return
            mid = len(arr) // 2               
                
            if node.value == None:
                node.value = arr[mid]
            else:
                node = TreeNode(value=arr[mid])
                
            node.left = insert(node, arr[:mid])
            node.right = insert(node, arr[mid+1:])
                
            return node
            
        arr = sorted(arr)
        self.root = TreeNode()
        processed = []
        # insert(self.root, arr, processed, 0, len(arr))
        insert(self.root, arr)
        return self.root

In [236]:
n4 = TreeNode(value=4)
n5 = TreeNode(value=5)
n2 = TreeNode(left=n4, right=n5, value=2)
n3 = TreeNode(value=3)
n1 = TreeNode(left=n2, right=n3, value=1)

tree = Tree(n1)

In [237]:
# (a) Inorder (Left, Root, Right) : 4 2 5 1 3
# (b) Preorder (Root, Left, Right) : 1 2 4 5 3
# (c) Postorder (Left, Right, Root) : 4 5 2 3 1
print(tree.inorder())
print(tree.preorder())
print(tree.postorder())

[4, 2, 5, 1, 3]
[1, 2, 4, 5, 3]
[1, 2, 4, 5, 3]


In [238]:
arr = [1,2,3,4,4,5,6,7,8,9]
btree = TreeGenerator(arr)
print(btree.inorder())

4.410743713378906e-05
[1, 2, 3, 4, 4, 5, 6, 7, 8, 9]


In [239]:
import math
import numpy as np

from Bio import SeqIO, AlignIO
from Bio.Align import MultipleSeqAlignment


In [240]:
with open("Coronavirus_Data.fasta", "rU") as handle:
    records = list(SeqIO.parse(handle, "fasta"))


In [241]:
epsilon = 2.2204e-16

def jukes_cantor(sequence1, sequence2):
    '''Calculate the Jukes-Cantor distance between the two provided aligned
    sequences.
    '''
    
    # Initialization
    difference_counter = 0
    length_counter = 0
    
    # Step 1: Count differences between sequences, ignoring gaps
    for i in range(min(len(sequence1), len(sequence2))):
        if sequence1[i] != '-' and sequence2[i] != '-':
            length_counter += 1
            if sequence1[i] != sequence2[i]:
                difference_counter += 1
    
    # Step 2: Calculate and return results
    p = difference_counter / length_counter
    jukes = -3.0/4.0 * math.log(max(epsilon, 1 - 4.0/3.0 * p))
    return jukes

In [242]:
align = MultipleSeqAlignment([])
for r in records:
    align.add_sequence(r.id, r.seq.__str__())

In [243]:
# dist_mat = np.zeros((len(align), len(align)))
# for i in range(len(align)):
#     for j in range(i+1, len(align)):
#         dist_mat[i][j] = jukes_cantor(align[i], align[j])

# dist_mat += dist_mat.T


In [244]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio.Phylo.TreeConstruction import DistanceCalculator

constructor = DistanceTreeConstructor()
calculator = DistanceCalculator('identity')
dm = calculator.get_distance(align)
upgmatree = constructor.upgma(dm)
print(upgmatree)
print(dm.names[np.argpartition(dm["Human_Sars_CoV"], 1)[1]])
print(dm.names[np.argpartition(dm["2019_nCoV_CDS1"], 1)[1]])

Tree(rooted=True)
    Clade(branch_length=0, name='Inner12')
        Clade(branch_length=0.004329156839516957, name='Inner9')
            Clade(branch_length=0.46680902169089356, name='Porcine_PEDV')
            Clade(branch_length=0.003363026099416744, name='Inner6')
                Clade(branch_length=0.14113494907984891, name='Inner3')
                    Clade(branch_length=0.3223110465116279, name='Murine_HV2')
                    Clade(branch_length=0.3223110465116279, name='Murine_HV1')
                Clade(branch_length=0.4634459955914768, name='Human_CoV_OC43')
        Clade(branch_length=0.0008431449802728808, name='Inner11')
            Clade(branch_length=0.47029503355013763, name='2019_nCoV_CDS1')
            Clade(branch_length=0.004288148868554165, name='Inner10')
                Clade(branch_length=0.003169945669587504, name='Inner7')
                    Clade(branch_length=0.46600688468158347, name='Feline_CoV4')
                    Clade(branch_length=0.4660068846815

In [245]:
dist_mat = dm.matrix.copy()
labels = dm.names.copy()
l = len(dist_mat[-1])
for i in range(l):
    dist_mat[i] += [0]*(l-i-1)
dist_mat = np.array(dist_mat)
dist_mat += dist_mat.T

In [246]:
import pandas as pd
labels = [r.id for r in align]
df = pd.DataFrame(dist_mat, columns=labels, index=labels)
pd.options.display.float_format = "{:,.2f}".format
df

,Bovine_CoV1,Bovine_CoV2,Human_CoV_OC43,Porcine_HEV3,Murine_HV1,Murine_HV2,IBV3,Porcine_PEDV,Feline_CoV4,Human_Sars_CoV,Palm_civet,2019_nCoV_CDS1,Bat_SARS-like_coronavirus
Bovine_CoV1,0.00,0.01,0.92,0.67,0.93,0.93,0.95,0.94,0.93,0.93,0.93,0.94,0.92
Bovine_CoV2,0.01,0.00,0.92,0.67,0.93,0.93,0.95,0.94,0.93,0.93,0.93,0.94,0.92
Human_CoV_OC43,0.92,0.92,0.00,0.92,0.92,0.93,0.95,0.93,0.94,0.94,0.94,0.94,0.94
Porcine_HEV3,0.67,0.67,0.92,0.00,0.93,0.93,0.95,0.94,0.93,0.93,0.93,0.95,0.91
Murine_HV1,0.93,0.93,0.92,0.93,0.00,0.64,0.94,0.93,0.93,0.96,0.96,0.95,0.94
Murine_HV2,0.93,0.93,0.93,0.93,0.64,0.00,0.95,0.94,0.94,0.95,0.95,0.93,0.94
IBV3,0.95,0.95,0.95,0.95,0.94,0.95,0.00,0.94,0.93,0.93,0.93,0.93,0.93
Porcine_PEDV,0.94,0.94,0.93,0.94,0.93,0.94,0.94,0.00,0.94,0.96,0.96,0.94,0.95
Feline_CoV4,0.93,0.93,0.94,0.93,0.93,0.94,0.93,0.94,0.00,0.94,0.94,0.95,0.94
Human_Sars_CoV,0.93,0.93,0.94,0.93,0.96,0.95,0.93,0.96,0.94,0.00,0.01,0.94,0.94


In [247]:
target = "Human_Sars_CoV"
t = dist_mat[labels.index(target),:]
closest = labels[np.argpartition(t, 1)[1]]
print(f"{closest} is closest to {target}")

Palm_civet is closest to Human_Sars_CoV


In [248]:
target = "2019_nCoV_CDS1"
t = dist_mat[labels.index(target),:]
closest = labels[np.argpartition(t, 1)[1]]
print(f"{closest} is closest to {target}")

Bat_SARS-like_coronavirus is closest to 2019_nCoV_CDS1
